# ArcGIS Online Web Map Audit

Description: This will audit all of the webmaps in an organization.
 
Created on: 2/13/2018
 
Purpose: The AGO Web Map Audit loops through all of the web maps in an organization and loops through the layers within the web map and creates an excel sheet that lists all of the map service urls within each web map, as well as, comparing the sharing status between the Webmap and Layers. 
 
Authored by: Joe Guzi  Contributions by: Rick Frantz
 
Previous Production Date: 6/30/24     

Production Date: 8/13/24

Notes:
- 6/30/24 --- Updated to capture content status of the item and the layers
- 8/13/24 --- updated with Audit Template

# Step 1: Import Libraries

In [ ]:
# import
print("Import Libraries...")
from arcgis.gis import GIS
import getpass
from IPython.display import display
from arcgis.mapping import WebMap
import json
import requests
import csv
from tkinter.filedialog import asksaveasfilename
print("Import Libraries Complete!")

# Step 2: Connect to ArcGIS Online or ArcGIS Enterprise

Use one of the methods below to conenct to ArcGIS Online or ArcGIS Enterprise.

## Connect with current AGO Credentials to this organization

Use this option if you are already signed into AGO and are running this notebook as a hosted notebook in AGO.

In [ ]:
gis = GIS("home")
gis
print("Connect to ArcGIS Online Complete!")

## Connect to ArcGIS Online by providing Org URL and Credentials (built-in users)

In [ ]:
# Connection Variables
Organization = input("What's your oranization? ")
User = input("What's your  username? ")
Password = getpass.getpass("What's your password? ")

# Connection 
gis = GIS(Organization, User, Password)
gis
print("Connected to AGO")

## Connect to ArcGIS Online by providing a profile

We connect by using a profile. This profile has been setup beforehand by using built-in credentials and registering it with the profile itself. We have a section in the documentation that shows you how to do this if you are interested.

In [ ]:
# Connection Variables
gis = GIS(profile="your_online_profile") # edit "your_online_profile" with the name of your profile
gis
print("Connected to AGO")

## User authentication with OAuth 2.0

https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/#user-authentication-with-oauth-20

In [ ]:
# Connection Variables
Organization = input("What's your organization? ")
OrgClientID = input("What's your  ClientID? ")
gis = GIS(Organization, client_id=OrgClientID)
print("Successfully logged in as: " + gis.properties.user.username)
print("Connected to AGO")

## Connect to ArcGIS Enterprise with Single Sign On

In [ ]:
# Connection Variables
Organization = input("What's your oranization? ")
gis = GIS(Organization)
gis
print("Logged in as " + gis.properties.user.username + " to " + gis.properties.portalName)
print("Connected to AGE")

# Step 3: Define Functions

In [ ]:
print("Begin Function Section.")
# This is where you define Functions
def CompareSharing(ItemAccess, LayerAccess):
    Issue = "No"
    
    if ItemAccess == "org" and LayerAccess == "private":
        Issue = "Yes"
    elif ItemAccess == "public" and LayerAccess == "private":
        Issue = "Yes"
    elif ItemAccess == "public" and LayerAccess == "org":
        Issue = "Yes"
    elif ItemAccess == "shared":
        Issue = "Maybe"
    elif LayerAccess == "shared":
        Issue = "Maybe"
    return Issue
print("Function Section Complete!")

# Step 4: Execute Code Section

In [ ]:
Audit = gis.content.search(query="", item_type="Web Map", max_items = 10000)
s = ""
    
for Item in Audit:
    ItemTitle = str(Item["title"].replace(',', ""))
    ItemID = str(Item["id"])
    print(ItemID)
    ItemType = str(Item["type"])
    ItemOwner = str(Item["owner"])
    ItemAccess = str(Item["access"])
    ItemContentStatus = str(Item.get("contentStatus"))
    ItemData = Item.get_data(try_json=True)
    for ItemInfo in ItemData:
        # Layers
        if ItemInfo == "operationalLayers":
            layers = ItemData["operationalLayers"]
            for layer in layers:
                #print (layer)
                s += ItemTitle.replace(",", "") + ","
                s += ItemID + ","
                s += ItemType + ","
                s += ItemOwner + ","
                s += ItemAccess + ","
                s += ItemContentStatus + ","
                s += "Operational Layer" + ","
                s += str(layer.get("id")) + ","
                s += str(layer.get("title")).replace(",", "") + ","
                s += str(layer.get("layerType")) + ","
                s += str(layer.get("visibility")) + ","
                LayerURL = str(layer.get('url'))
                s += LayerURL + ","
                LayerItemID = str(layer.get('itemId'))
                if LayerItemID != "None":
                    s += LayerItemID + ","
                    LayerItem = gis.content.search(query='id:' + str(LayerItemID))
                    if LayerItem != []:
                        LayerContentStatus = str(LayerItem[0].get("contentStatus"))
                        s += LayerContentStatus + ","
                        LayerAccess = str(LayerItem[0].get("access"))
                        s += LayerAccess + ","
                        s += "Yes" + ","
                        SharingIssue = CompareSharing(ItemAccess, LayerAccess)
                        s += SharingIssue + ","
                    else:
                        s += ","
                        s += ","
                        s += "No" + ","
                        s += "Maybe" + ","
                else:
                    s += ","
                s += "\n"
        # Tables
        if ItemInfo == "tables":
            tables = ItemData["tables"]
            for table in tables:
                #print (table)
                s += ItemTitle.replace(",", "") + ","
                s += ItemID + ","
                s += ItemType + ","
                s += ItemOwner + ","
                s += ItemAccess + ","
                s += ItemContentStatus + ","
                s += "Table" + ","
                s += str(table.get("id")) + ","
                s += str(table.get("title")).replace(",", "") + ","
                s += ","
                s += "," 
                s += str(table.get("url")) + "," #"" + ","
                LayerItemID = str(table.get('itemId'))
                if LayerItemID != "None":
                    s += LayerItemID + ","
                    LayerItem = gis.content.search(query='id:' + str(LayerItemID))
                    if LayerItem != []:
                        LayerContentStatus = str(LayerItem[0].get("contentStatus"))
                        s += LayerContentStatus + ","
                        LayerAccess = str(LayerItem[0].get("access"))
                        s += LayerAccess + ","
                        s += "Yes" + ","
                        SharingIssue = CompareSharing(ItemAccess, LayerAccess)
                        s += SharingIssue + ","
                    else:
                        s += ","
                        s += ","
                        s += "No" + ","
                        s += "Maybe" + ","
                else:
                    s += ","
                s += "\n"
        # Basemap
        if ItemInfo == "baseMap":
            basemaps = ItemData["baseMap"]
            BaseMapLayers = basemaps["baseMapLayers"]
            for basemaplayer in BaseMapLayers:
                #print (basemaplayer)
                s += ItemTitle.replace(",", "") + ","
                s += ItemID + ","
                s += ItemType + ","
                s += ItemOwner + ","
                s += ItemAccess + ","
                s += ItemContentStatus + ","
                s += "Basemap" + ","
                s += str(basemaplayer.get("id")) + ","
                s += str(basemaplayer.get("title")).replace(",", "") + ","
                s += str(basemaplayer.get("layerType")) + ","
                s += "," 
                s += str(basemaplayer.get("url")) + ","
                LayerItemID = str(basemaplayer.get('itemId'))
                if LayerItemID != "None":
                    s += LayerItemID + ","
                    LayerItem = gis.content.search(query='id:' + str(LayerItemID))
                    if LayerItem != []:
                        LayerContentStatus = str(LayerItem[0].get("contentStatus"))
                        s += LayerContentStatus + ","
                        LayerAccess = str(LayerItem[0].get("access"))
                        s += LayerAccess + ","
                        s += "Yes" + ","
                        SharingIssue = CompareSharing(ItemAccess, LayerAccess)
                        s += SharingIssue + ","
                    else:
                        s += ","
                        s += ","
                        s += "No" + ","
                        s += "Maybe" + ","
                else:
                    s += ","
                s += "\n"
                
                
print ("Audit Section Complete!")

# Step 5: Write the Results to Excel (Optional)

In [ ]:
print("Begin Write Results to Excel Section.") 
filename = asksaveasfilename() # show an "Open" dialog box and return the path to the selected file
print(filename)
csvfile = filename + ".csv"
print(csvfile)
with open(csvfile, 'w') as output:
    print ("Write data To CSV File")
    output.write("MapTitle,MapId,ItemType,ItemOwner,ItemAccess,ItemContentStatus,MapLocation,LayerID,LayerTitle,LayerType,LayerVisisbility,LayerURL,ItemID,LayerContentStatus,LayerAccess,OrganizationalLayer,SharingIssue\n")
    output.writelines(s)
    print("Write data To CSV File Complete!")

# Step 6: Upload to AGO (Optional)

In [ ]:
print("Begin Upload to AGO Section.") 
file = filename.rpartition('/')
testfile = gis.content.search(query="title:"+file[2], item_type="Feature Layer Collection")
if testfile:
    print("Found the service")
    testfile = testfile[0]
    display(testfile)
    from arcgis.features import FeatureLayerCollection
    testfile_collection = FeatureLayerCollection.fromitem(testfile)
    print("The table already exists")
    testfile_collection.manager.overwrite(csvfile)
    display
else:
    print("We have to upload it")
    csv_item = gis.content.add({}, csvfile)
    display(csv_item)
    csv_lyr = csv_item.publish()
    display(csv_lyr)
print ("The CSV has been published.\nAll done, thanks for playing!")